load data


In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('datasets/homedecor.csv')
df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Cost_Price,UnitPrice,Profit,Quantity,CustomerID,Country
0,975092,2023-02-20,401,couch,15000,25000,10000,1,1,India
1,975092,2023-02-20,407,bedsheet,500,1000,500,1,1,India
2,724424,2023-02-20,417,door knob,200,400,200,1,2,India
3,724424,2023-02-20,401,couch,15000,25000,10000,1,2,India
4,724424,2023-02-20,416,door,5000,10000,5000,1,2,India


let's cheak data and missing value and handle it

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   InvoiceNo    2551 non-null   int64 
 1   InvoiceDate  2551 non-null   object
 2   StockCode    2551 non-null   int64 
 3   Description  2551 non-null   object
 4   Cost_Price   2551 non-null   int64 
 5   UnitPrice    2551 non-null   int64 
 6   Profit       2551 non-null   int64 
 7   Quantity     2551 non-null   int64 
 8   CustomerID   2551 non-null   int64 
 9   Country      2551 non-null   object
dtypes: int64(7), object(3)
memory usage: 199.4+ KB


In [5]:
df['CustomerID'].nunique()

500

In [6]:
df.sort_values(by='InvoiceDate')

,InvoiceNo,InvoiceDate,StockCode,Description,Cost_Price,UnitPrice,Profit,Quantity,CustomerID,Country
0,975092,2023-02-20,401,couch,15000,25000,10000,1,1,India
16,957148,2023-02-20,402,lamp,500,750,250,1,5,India
15,957148,2023-02-20,403,tube light,450,600,150,1,5,India
14,957148,2023-02-20,404,queen bed,29000,45000,16000,1,5,India
13,957148,2023-02-20,406,pillow,200,400,200,1,5,India
...,...,...,...,...,...,...,...,...,...,...
2545,149536,2024-02-20,403,tube light,450,600,150,1,261,India
2546,149536,2024-02-20,409,shelf,500,1000,500,1,261,India
2547,837372,2024-02-20,415,yard chair,1500,3000,1500,1,421,India
2548,837372,2024-02-20,403,tube light,450,600,150,1,421,India


 we have missing value in 'CustomerID'and 'Description'
 drop it is a good option to handle it
 also I will handle duplicates rows


In [7]:
df=df.dropna(axis=0)
df=df.drop_duplicates()

'InvoiceDate'is an object columns ,let's convert it to datetime and get days

In [8]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
#to convert the columns to days ,we need to get the last day cheaked in data
last_date=df['InvoiceDate'].max()
df['InvoiceDate']=last_date-df['InvoiceDate']
df['InvoiceDate']=df['InvoiceDate'].dt.days

**RFM analysis**

In [9]:

df['the amount']=df['Quantity']*df['UnitPrice']

recency,frequency,monetary

In [10]:
r_df=df.groupby('CustomerID')['InvoiceDate'].min()
f_df=df.groupby(df['CustomerID'])['InvoiceNo'].count()
m_df=df.groupby(df['CustomerID'])['the amount'].sum()

In [11]:
rf_data=pd.merge(r_df,f_df,on='CustomerID', how='inner')
rfm_data=pd.merge(rf_data,m_df,on='CustomerID', how='inner')
rfm_data.head()

,InvoiceDate,InvoiceNo,the amount
CustomerID,,,
1,95,8,46800
2,365,5,53400
3,5,6,33150
4,50,10,110850
5,86,10,105650


In [12]:
country_data=df.groupby(df['Country'])['CustomerID'].count()
country_data

Country
India    2551
Name: CustomerID, dtype: int64

United kingdom is most freq country ,so let's use it to make analysis

In [13]:
id_coun=df[['Country','CustomerID']]
rfmc_data=pd.merge(rfm_data,id_coun,on='CustomerID', how='left')
rfmc_data=rfmc_data.drop_duplicates()
rfmc_data=rfmc_data.set_index('CustomerID')
rfmc_data=rfmc_data.rename(columns={'InvoiceDate':'recency',
                                    'InvoiceNo':'frequency',
                                    'the amount':'monetary'})

rfmc_data.groupby(rfmc_data['Country']).count()
UK_rfmc_data=rfmc_data[rfmc_data['Country']=='India']
# UK_rfmc_data = UK_rfmc_data[UK_rfmc_data['recency'] != 0]

UK_rfmc_data.head()

,recency,frequency,monetary,Country
CustomerID,,,,
1,95,8,46800,India
2,365,5,53400,India
3,5,6,33150,India
4,50,10,110850,India
5,86,10,105650,India


**it possible to make analysis to each country ,but in this notebook UK is a good choice to make analysis**

now I will handle outliers data


In [ ]:
sns.boxplot(UK_rfmc_data['recency'])

KeyError: 0

In [ ]:
sns.boxplot(UK_rfmc_data['frequency'])


In [ ]:
sns.boxplot(UK_rfmc_data['monetary'])

in each columns, thier are outliers data

In [14]:
UK_rfmc_data.describe()

,recency,frequency,monetary
count,500.000000,500.000000,500.00000
mean,154.508000,5.102000,59392.50000
std,112.327061,3.155235,55043.44266
min,0.000000,1.000000,100.00000
25%,51.000000,3.000000,13150.00000
50%,143.000000,4.000000,47000.00000
75%,247.250000,7.000000,90000.00000
max,365.000000,17.000000,299950.00000


in 'recency' column their are 0 value ,in 'monetary' column thier are negative value,so handle it by log tranformation is not possible

now I will delete it

In [15]:
def remove_outlier(df, col_name):
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3-q1
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df = df[(df[col_name] > fence_low) & (df[col_name] < fence_high)]
    return df
L=0
while L<=3:
    select_col=UK_rfmc_data.columns[L]
    remove_data=remove_outlier(UK_rfmc_data,select_col)
    UK_rfmc_data=remove_data
    print('turn num ',L+1)
    print(UK_rfmc_data.shape)
    L+=1
    if L==3:
        break

UK_rfmc_data.head()

turn num  1
(500, 4)
turn num  2
(485, 4)
turn num  3
(474, 4)


,recency,frequency,monetary,Country
CustomerID,,,,
1,95,8,46800,India
2,365,5,53400,India
3,5,6,33150,India
4,50,10,110850,India
5,86,10,105650,India


In [16]:
#rfm score
quantiles = UK_rfmc_data.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

def RScoring(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

def FnMScoring(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

UK_rfmc_data['R'] =UK_rfmc_data['recency'].apply(RScoring,
                                            args=('recency',quantiles,))
UK_rfmc_data['F'] = UK_rfmc_data['frequency'].apply(FnMScoring,
                                              args=('frequency',quantiles,))
UK_rfmc_data['M'] =UK_rfmc_data['monetary'].apply(FnMScoring,
                                             args=('monetary',quantiles,))


UK_rfmc_data['RFM SCORE']=UK_rfmc_data[['R','F','M']].sum(axis=1)
UK_rfmc_data

C:\Users\Monkey.D.Luffy\AppData\Local\Temp\ipykernel_13912\1769169022.py:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  quantiles = UK_rfmc_data.quantile(q=[0.25,0.5,0.75])


,recency,frequency,monetary,Country,R,F,M,RFM SCORE
CustomerID,,,,,,,,
1,95,8,46800,India,2,1,2,5
2,365,5,53400,India,4,2,2,8
3,5,6,33150,India,1,2,3,6
4,50,10,110850,India,1,1,1,3
5,86,10,105650,India,2,1,1,4
...,...,...,...,...,...,...,...,...
496,50,9,31400,India,1,1,3,5
497,33,7,74350,India,1,2,2,5
498,160,3,49100,India,3,3,2,8


now ,I will make Loyalty Level columns ,then i will split data to 4 dataframe to make handle it more easy

In [17]:
S_customer=UK_rfmc_data[UK_rfmc_data['RFM SCORE']==3]
S_customer['RFM_Loyalty_Leve']='S'

UK_rfmc_data=UK_rfmc_data[UK_rfmc_data['RFM SCORE']!=3]

Loyalty_Level = ['A','B','C']
Score_cuts = pd.qcut(UK_rfmc_data['RFM SCORE'], q = 3, labels = Loyalty_Level)
UK_rfmc_data['RFM_Loyalty_Level'] = Score_cuts.values

A_customer=UK_rfmc_data[UK_rfmc_data['RFM_Loyalty_Level']=='A']
B_customer=UK_rfmc_data[UK_rfmc_data['RFM_Loyalty_Level']=='B']
C_customer=UK_rfmc_data[UK_rfmc_data['RFM_Loyalty_Level']=='C']


C:\Users\Monkey.D.Luffy\AppData\Local\Temp\ipykernel_13912\2499275944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S_customer['RFM_Loyalty_Leve']='S'
C:\Users\Monkey.D.Luffy\AppData\Local\Temp\ipykernel_13912\2499275944.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UK_rfmc_data['RFM_Loyalty_Level'] = Score_cuts.values


In [18]:
merged_df = pd.merge(df, UK_rfmc_data[['R', 'F', 'M']], left_on='CustomerID', right_index=True, how='left')
merged_df

,InvoiceNo,InvoiceDate,StockCode,Description,Cost_Price,UnitPrice,Profit,Quantity,CustomerID,Country,the amount,R,F,M
0,975092,365,401,couch,15000,25000,10000,1,1,India,25000,2.0,1.0,2.0
1,975092,365,407,bedsheet,500,1000,500,1,1,India,1000,2.0,1.0,2.0
2,724424,365,417,door knob,200,400,200,1,2,India,400,4.0,2.0,2.0
3,724424,365,401,couch,15000,25000,10000,1,2,India,25000,4.0,2.0,2.0
4,724424,365,416,door,5000,10000,5000,1,2,India,10000,4.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,149536,0,409,shelf,500,1000,500,1,261,India,1000,1.0,2.0,1.0
2547,837372,0,415,yard chair,1500,3000,1500,1,421,India,3000,NaN,NaN,NaN
2548,837372,0,403,tube light,450,600,150,1,421,India,600,NaN,NaN,NaN
2549,591706,0,415,yard chair,1500,3000,1500,1,279,India,3000,1.0,2.0,1.0


In [19]:
# Define shelf capacities
shelf_capacities = [100, 200, 300, 400, 500]

# Sort products by RFM scores
merged_df = merged_df.sort_values(by=['R', 'F', 'M'], ascending=False)

# Initialize a dictionary to store shelf allocations
shelf_allocations = {}

# Initialize a set to keep track of allocated products
allocated_products = set()

# Iterate through products and allocate to shelves
for index, row in merged_df.iterrows():
    for shelf_id, capacity in enumerate(shelf_capacities):
        if row['Description'] not in allocated_products and row['Quantity'] <= capacity:
            shelf_allocations.setdefault(shelf_id, []).append(row['Description'])
            shelf_capacities[shelf_id] -= row['Quantity']
            allocated_products.add(row['Description'])
            break

# Print shelf allocations
for shelf_id, products in shelf_allocations.items():
    print(f"Shelf {shelf_id+1}: {', '.join(products)}")


Shelf 1: yard chair, pillow, side table, paint, curtain, tube light, lamp, carpet, chair, tiles, door, door knob, bedsheet, shelf, area rug, couch, cupboard, dining table, queen bed


In [20]:
# lowest RF scores = STOCK CLEARANCE
merged_df['RF_Sum'] = merged_df['R'] + merged_df['F']

merged_df_sorted = merged_df.sort_values(by='RF_Sum', ascending=True)
lowest_rf_product = merged_df_sorted.iloc[0]

print("Product with the lowest RF score:")
print(lowest_rf_product)
print(lowest_rf_product.Description)

Product with the lowest RF score:
InvoiceNo      720736
InvoiceDate         3
StockCode         406
Description    pillow
Cost_Price        200
UnitPrice         400
Profit            200
Quantity            1
CustomerID        187
Country         India
the amount        400
R                 1.0
F                 1.0
M                 2.0
RF_Sum            2.0
Name: 2512, dtype: object
pillow


In [21]:
# highest MF scores = PRODUCT OF THE MONTH
merged_df['MF_Sum'] = merged_df['M'] + merged_df['F']

merged_df_sorted2 = merged_df.sort_values(by='MF_Sum', ascending=False)
highest_mf_product = merged_df_sorted2.iloc[0]

print("Product with the highest MF score:")
print(highest_mf_product)
print(highest_mf_product.Description)

Product with the highest MF score:
InvoiceNo          574066
InvoiceDate           358
StockCode             415
Description    yard chair
Cost_Price           1500
UnitPrice            3000
Profit               1500
Quantity                1
CustomerID             23
Country             India
the amount           3000
R                     4.0
F                     4.0
M                     4.0
RF_Sum                8.0
MF_Sum                8.0
Name: 77, dtype: object
yard chair


In [22]:
# highest MF scores = SEASONAL
merged_df['RM_Sum'] = merged_df['R'] + merged_df['M']

merged_df_sorted3 = merged_df.sort_values(by='RM_Sum', ascending=False)
highest_rm_product = merged_df_sorted3.iloc[0]

print("Product with the highest RM score:")
print(highest_rm_product)
print(highest_rm_product.Description)

Product with the highest RM score:
InvoiceNo          574066
InvoiceDate           358
StockCode             415
Description    yard chair
Cost_Price           1500
UnitPrice            3000
Profit               1500
Quantity                1
CustomerID             23
Country             India
the amount           3000
R                     4.0
F                     4.0
M                     4.0
RF_Sum                8.0
MF_Sum                8.0
RM_Sum                8.0
Name: 77, dtype: object
yard chair


In [23]:
merged_df['R'].mean() + merged_df['M'].mean() + merged_df['F'].mean()

6.687050359712229